<a href="https://colab.research.google.com/github/Jeanhenry/datascience/blob/master/Analisando_os_Dados_do_Airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="15%" src="https://i.ibb.co/2Zy6D4y/logo-colab.png" alt="logo-colab" border="0">

#### **Data Science na Prática 2.0**
*by [jean henry](www.linkedin.com/in/jeanhenry)*

---


<img alt="Analisando Airbnb" width="10%" src="https://www.area360.com.au/wp-content/uploads/2017/09/airbnb-logo.jpg">

# Análise dos Dados do Airbnb - *Rio de Janeiro/RJ*

O [Airbnb](https://www.airbnb.com.br/) é uma das pioneiras nesse ramo de "aluguel de imóveis", indo de frente com muitos hotéis, ela se tornou referencia.

Um meio mais fácil e mais pratico _(mas não quer dir que seja seguro)_ de viajar e ter um "canto pra ficar".

Uma das iniciativas do Airbnb é disponibilizar dados do site, para algumas das principais cidades do mundo. Por meio do portal [Inside Airbnb](http://insideairbnb.com/get-the-data.html), é possível baixar uma grande quantidade de dados para desenvolver projetos e soluções de *Data Science*.

**Neste *notebook*, iremos analisar os dados referentes à cidade Rio de Janeiro, e ver quais insights podem ser extraídos a partir de dados brutos.**

## Obtenção dos Dados

Todos os dados dessa análise, é possivel baixar no proprio site da Airbnb.

Para essa análise, estaremos usando apenas o arquivo:

* _listing.csv_ da cidade do Rio de Janeiro/RJ


In [0]:
# importar os pacotes necessarios
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
# importar o arquivo listings.csv para um DataFrame
df = pd.read_csv("http://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2020-04-20/visualisations/listings.csv")

## Análise dos Dados

Abaixo vamos detalhar todos as variaveis deste dataset, para que possamos entender melhor a nossa base de dados.😉


**Dicionário das variáveis**

* _id_ > número de id gerado pelo proprio Airbnb para identificar o imóvel
* _name_ > nome do imóvel em questão
* _host_id_ > número de id do proprietário do imóvel
* _host_name_ > Nome do proprietário do imóvel
* _neighbourhood_group_ > coluna vazia, sem informação para análise
* _neighbourhood_ > nome do bairro do imóvel
* _latitude_ > coordenada geográfica do impovel 
* _longitude_ > coordenada geográfica do impovel
* _room_type_ > tipo de quarto oferecido no anuncio
* _price_ > valor do aluguel
* _minimun_nights_ > quantidade minima de noites para se hospedar
* _number_of_reviews_ > número reviews do imóvel
* _last_review_ > data do último review do imóvel
* _reviews_per_month_ > quantidade de reviews por mês
* _calculated_host_listings_count_ > quantidade de imóveis do mesmo proprietário
* _availability_365_ > número de disponibilidade dentro de 365 dias


Agora vamos dar um confere em nosso dataset pra entender melhor nosso *dataset*, de imediato vamos listar apenas as 5 primeiras entradas.

In [0]:
# mostrar as 5 primeiras entradas
df.head()

### **Q1. Quantos atributos (variáveis) e quantas entradas o nosso conjunto de dados possui? Quais os tipos das variáveis?**

Para essa análise inicial, estaremos usando um Dataset resumido, mas no próprio site onde baixamos esse, possui um mais completo com +-35.847 registros(entradas) e +-106 colunas (variaveis)

In [0]:
# identificar o volume de dados do DataFrame
print('No DataSet em questão, temos',df.shape[0],'registros.') #numero de registros/entradas
print('No DataSet em questão, temos',df.shape[1],'colunas.','\n') #numero de colunas/variavies

# verificar as 5 primeiras entradas do dataset [PLUS]
print(df.dtypes,'\n')

# verificar as 5 primeiras entradas do dataset
df.head()

### **Q2. Qual a porcentagem de valores ausentes no *dataset*?**

Para qualquer análise, é de suma importância entender o real cenário do dataset, quando digo isso quero dizer, entender se o dataset possui dados **NULOS**, dados inconsistentes e se esse dados vão afetar em nossa análise ou não, caso contrário, devemos tratar esses dados.

**Conclusão**
* Das 16 colunas disponiveis no dataset:
  * mais da metade delas possui 0% de _"dados nulos"_ (isso é bom mas não quer dizer nada ainda, devemos ver se esses dados são ou possuem outliers)
  * 2 colunas (reviews_per_month e last_review), possuem 40% de "dados nulos"
  * A coluna neighbourhood_group, possui 100% de "dados nulos" (se essa coluna não agregar em nossa análise, podemos eliminar)

In [0]:
# ordenar em ordem decrescente as variáveis por seus valores ausentes

(df.isnull().sum() / df.shape[0]).sort_values(ascending=False)

### **Q3. Qual o tipo de distribuição das variáveis?** 


In [0]:
# plotar o histograma das variáveis numéricas
df.hist(bins=15, figsize=(15,10));

**+ ANÁLISE**

No histograma acima, conseguimos ver claramente que existe um ou mais _outliers_.
* O preço do imóvel (price) 
* Mínimo de noites (minimun_nights)

_Esses outliers devem ser tratados, caso contrário nos retorna informações inconcistentes e com isso a nossa análise vai estar incorreta_

Para tirarmos a prova dos 9 se existe outliers, vamos realizar 2 visões:

  * Usando o _describe()_
  * Plotando um _boxplots_ para cada variavel que acreditamos ter ums distorção de valores.

In [0]:
# ver resumo estatistico das variaveis numericas

df[['price','minimum_nights','number_of_reviews','reviews_per_month',
    'calculated_host_listings_count','availability_365']].describe()

Viu só, até que não estavamos tão enganado assim, vejas as colunas e me diz se não tem algo estranho? Mesmo não sendo e caso não conheça o estado do _RIO DE JANEIRO_

**_Analisando a coluna price_**
* Existe 1 ou + imóveis que não possuem valores de aluguel.
* Olha o valor máximo do aluguel. Esta na faixa de R$131.727,00... Onde sera  essa moradia em?rs

**_Analisando a coluna minimum_nights_**
* Sera que tem alguem que fique + de 1 ano hospedado pelo Airbnb?

E por ai vai, seguindo essa análise conseguimos entender com mais clareza nossas variáveis.

Agora vamos plotar um _boxplots_ só pra fecharmos nossa análise com chave de ouro.


In [0]:
#price
df.minimum_nights.plot(kind='box', vert=False, figsize=(15,3))
plt.show


#ver a qtd de valores acima de R$1.500 para price
print("price: valores acima de R$1.500")
print('Entradas:',len(df[df.price > 1500]))
print((len(df[df.price > 1500]) / df.shape[0]) * 100,'\n')

**FINALIZANDO A VISÃO DE OUTLIERS**

Conseguimos identificar quais são os nossos outliers, vamos tratar nosso Dataset

In [0]:
#remover os outliers em um novo df
df_clean = df.copy()
df_clean.drop(df_clean[df_clean.price > 1500].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.minimum_nights > 30].index, axis=0, inplace=True)

#remover a coluna neighbourhood_group, pois esta 100% nulo
df_clean.drop('neighbourhood_group',axis=1, inplace=True)

#plotar um novo histograma mas LIMPO sem os outliers e a coluna 100% NULA
df_clean.hist(bins=15, figsize=(15,10));


In [0]:
#minimun_nights
df.minimum_nights.plot(kind='box', vert=False, figsize=(15,3))
plt.show

#ver a qtd de valores acima de 30 dias para minimum_nights
print("minimum_nights: valores acima de 30 dias")
print('Entradas:',len(df[df.minimum_nights > 30]))
print((len(df[df.minimum_nights > 30]) / df.shape[0]) * 100,'\n')

### **Q4. Qual a média dos preços de aluguel?**

In [0]:
# ver a média da coluna `price``
df['price'].describe()

#Conseguimos identificar em mean, que nesse caso é 753.686367

### **Q4. Qual a correlação existente entre as variáveis**

In [0]:
# criar uma matriz de correlação
corr = df_clean[['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
    'calculated_host_listings_count', 'availability_365']].corr()

# mostrar a matriz de correlação
display(corr)

# plotar um heatmap a partir das correlações
sns.heatmap(corr, cmap='RdBu', fmt='.2f', square=True, linecolor='white', annot=True);


### **Q5. Qual o tipo de imóvel mais alugado no Airbnb?**

Nessa análise podemos identificar alguns tipos de aluguéis, como:
* apartamentos
* casas inteiras
* aluguel de 1 quarto
* dividir quarto com outras pessoas

In [0]:
# mostrar a quantidade de cada tipo de imóvel disponível

df_clean.room_type.value_counts()

In [0]:
# mostrar a porcentagem de cada tipo de imóvel disponível

df_clean.room_type.value_counts() / df_clean.shape[0]

### **Q6. Qual a localidade mais cara do dataset?**

Pra essa análise, usaremos a seguinte variavel _neighbourhood_.



In [0]:
# ver preços por bairros, na média
df_clean.groupby(['neighbourhood']).price.mean().sort_values(ascending=False)[:10]


**OBSERVAÇÃO**

Chegamos um ponto interresante de nossa análise, pra quem conhece o estado do RIO DE JANEIRO, sabe que o bairro de Vaz Lobo fica situado na Zona NORTE e onde bem sabemos a Zona SUL, de qualquer lugar se trata do lugar + caro com relação a moradia, alimentação e demais itens (chamamos de custo benefício), com isso, ja iria ligar o alerta ao ver esse cenário, mas para quem não conhece o RJ, ficaria meio complicado entender isso (+-), pois sabemos que os bairros de **COPACABANA, LEBLON, IPANEMA, BARRA DA TIJUCA** nem aparece nesse visual. Esta ai o diferencial de quem estiver realizando a análise de um dataset, entender se o que esta retornando para nós é de fato o que acontece na atualidade, vamos entender melhor esse resultado abaixo, realizando uma pequena análise + profunda e simples.

In [0]:
#Qtd de imoveis de Vaz Lobo
print(df_clean[df_clean.neighbourhood == "Vaz Lobo"].shape)

#ver as 3 entradas de Vaz Lobo
df_clean[df_clean.neighbourhood == "Vaz Lobo"]

Com base na análise acima, podemos identificar que no bairro **Vaz Lobo** existem apenas 3 registros de entrada, onde 1 deles com o valor alto.

In [0]:
# plotar os imóveis pela latitude-longitude
df_clean.plot(kind="scatter", x='longitude', y='latitude',alpha=0.4,c=df_clean['price'],s=8,cmap=plt.get_cmap('jet'),figsize=(12,8));

### **Q7. Qual é a média do mínimo de noites para aluguel (minimum_nights)?**

In [0]:
# ver a média da coluna `minimum_nights``
df['minimum_nights'].describe()

#Conseguimos identificar em mean, que nesse caso é 4.781851

In [0]:
#maiores bairros com imoveis alugados

df_clean.neighbourhood.value_counts()

## Conclusões


Chegamos ao final de uma análise bem ampla, onde conseguimos identificar por detalhes alguns itens _(destes até ja tinhamos uma total certeza pelo nosso achismo)_.

* A média de aluguéis no estado do RJ é de **753.68**
* Nesse DataSet , vimos que o Bairro com o valor do aluguel mais caro esta localizado na Zona Norte do RJ [VAZ LOBO], mas com nossa análise mais apurada identificamos que houve um "erro" na variável preço, pois para esse bairro temos apenas 3 imóveis disponíveis.
* Existe uma grande aglomeração de aluguel próximos a praia do RJ (ja era de ser esperado né?)
* No TOP 3 de imóveis a serem alugados todos estão localizados na Zona SUL _(mais uma certeza se concretizando)_


